# Reading in Cleaned data & dictionary

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive")

Mounted at /content/drive/


In [ ]:
import openpyxl

xlsx_filename = 'Retrain-Lemmatized.xlsx'
ner_dict = {}

# Load the Excel file
workbook = openpyxl.load_workbook(xlsx_filename)

# Assuming data is in the first sheet (you can specify the sheet name if different)
sheet = workbook.active

# Iterate through rows in the sheet
for row in sheet.iter_rows(min_row=2, values_only=True):
    tag, word = row[0], row[1]

    # Convert the 'word' to lowercase
    word = word.lower()

    if tag in ner_dict:
        ner_dict[tag].append(word)
    else:
        ner_dict[tag] = [word]

# Print the resulting dictionary
ner_dict


{'DOC': ['audiologist',
  'ent',
  'primary care doctor',
  'physical therapy',
  'neurologist',
  'doctor',
  'pc',
  'physical thereapist',
  'cardiologist',
  'chiropractor',
  'pediatrician',
  'ear nose throat',
  'specialist',
  'otolaryngologist',
  'general practitioner',
  'gp',
  'psychologist',
  'counselor',
  'therapist'],
 'DIAG': ['cochlear hyrdrops',
  'eustactian tube dysfunction',
  'tmj',
  'dysfunction',
  'tumor',
  'meniere',
  'hydrops',
  'trauma',
  'middle ear',
  'epilepsy',
  'ptsd',
  'somatic',
  'rupture',
  'sensorineural',
  'cochlear',
  'hyperacusis',
  'sinusitis',
  'post nasal drip'],
 'TEST': ['otoscopy',
  'neck examination',
  'auscultations',
  'clinical examination',
  'audiometry',
  'tympanometry',
  'oae',
  'psychiatric',
  'eeg',
  'mri',
  'ct',
  'echo-doppler',
  'baep',
  'cardiovascular examination',
  'psychiatric',
  'angiography',
  'cardiovascular examination',
  'imaging',
  'blood test',
  'doppler'],
 'MDIAG': ['misdiagnose'],

In [ ]:
from collections import Counter

xlsx_filename2 = 'Reddit-Stories-Split.xlsx'
dictionary_data = {}

# Load the Excel file
workbook = openpyxl.load_workbook(xlsx_filename2)

# Assuming data is in the first sheet (you can specify the sheet name if different)
sheet = workbook.active
id = 1
# Iterate through rows in the sheet
for row in sheet.iter_rows(min_row=2, values_only=True):
  if row[0] != None:
    s = row[0].lower().split()
    if len(s) > 25:
      s = row[0].lower().split()[:25]
    for i in range(len(s)):
      matched = False  # Flag to check if the word was matched
      for key, value in ner_dict.items():
          if s[i] in value:
              s[i] = key
              matched = True
              break  # Exit the inner loop if a match is found
      if not matched:
          s[i] = '0'  # Set a placeholder if the word wasn't matched

    # Filter the list to remove 0s
    filtered_list = [x for x in s if x != '0']

    # Use Counter to count occurrences
    if (len(filtered_list)) > 0:
      element_counts = Counter(filtered_list)
      # Find the most common element and its count
      most_common_element, count = element_counts.most_common(1)[0]
    else:
      most_common_element = None

    dictionary_data[id] = [row[0], most_common_element]
    id += 1


len(dictionary_data)

555

In [ ]:
'''for k,v in dictionary_data.items():

  if v[1] in classification['main']:
    dictionary_data[k].append('main')
  elif v[1] in classification['note']:
    dictionary_data[k].append('note')
  elif v[1] in classification['emotion']:
    dictionary_data[k].append('emotion')
  else:
    dictionary_data[k].append('none')'''

KeyError: ignored

In [ ]:
none = 0
main = 0
note = 0
for k,v in dictionary_data.items():
  if v[2] == 'none':
    none +=1
  if v[2] == 'main':
    main +=1
  if v[2] == 'note':
    note +=1

f"there are {none} none classes, {main} main classes and {note} note classes"

IndexError: ignored

In [ ]:
import pandas as pd

df = pd.DataFrame(dictionary_data)

df = df.T

df.head()

,0,1
1,"Ive had tinnitus for 4 years, and it developed...",SYM
2,I had a lot of fullness and loud ringing in my...,SOUND
3,The ringing was only loud for maybe a month or...,SOUND
4,"Ive had a few ear infections since then, and i...",None
5,"Fast forward to last week, I went and visited ...",None


In [ ]:
# Rename multiple columns
df = df.rename(columns={0: 'Sentences', 1: 'Most Frequent NER', 2: 'Class'})

df.head()


,Sentences,Most Frequent NER
1,"Ive had tinnitus for 4 years, and it developed...",SYM
2,I had a lot of fullness and loud ringing in my...,SOUND
3,The ringing was only loud for maybe a month or...,SOUND
4,"Ive had a few ear infections since then, and i...",None
5,"Fast forward to last week, I went and visited ...",None


In [ ]:
df.to_csv('Reddit-Classification.csv')

# NER Model

In [ ]:
ner_dict

df.head()

,Sentences,Most Frequent NER
1,"Ive had tinnitus for 4 years, and it developed...",SYM
2,I had a lot of fullness and loud ringing in my...,SOUND
3,The ringing was only loud for maybe a month or...,SOUND
4,"Ive had a few ear infections since then, and i...",None
5,"Fast forward to last week, I went and visited ...",None


In [ ]:
def assign_ner_tags(sentence):
    tagged_words = []
    start = 0  # Initialize the start position

    for token in sentence.split():
        for ner_tag, tag_words in ner_dict.items():
            if token.lower() in tag_words:
                end = start + len(token)  # Calculate the end position
                tagged_words.append((token, ner_tag, start, end))
            start += len(token) + 1  # Increment the start position for the next token and account for the space

    return tagged_words

df['NER Tags'] = df['Sentences'].apply(assign_ner_tags)
df.head()


,Sentences,Most Frequent NER,NER Tags
1,"Ive had tinnitus for 4 years, and it developed...",SYM,"[(tinnitus, SOUND, 132, 140), (infection, SYM,..."
2,I had a lot of fullness and loud ringing in my...,SOUND,"[(ringing, SOUND, 428, 435)]"
3,The ringing was only loud for maybe a month or...,SOUND,"[(ringing, SOUND, 80, 87), (infection, SYM, 92..."
4,"Ive had a few ear infections since then, and i...",None,[]
5,"Fast forward to last week, I went and visited ...",None,[]


In [ ]:
def assign_ner_tags(sentence):
    tagged_words = []
    start = 0

    for token in sentence.split():
        for ner_tag, tag_words in ner_dict.items():
            if token.lower() in tag_words:
                end = start + len(token)  #Calculate the end position
                tagged_words.append((token, ner_tag, start, end))
        start += len(token) + 1  #Increment the start position for the next token and account for the space

    return tagged_words

# Apply the function to the DataFrame
df['NER Tags'] = df['Sentences'].apply(assign_ner_tags)


In [ ]:
df.head()

,Sentences,Most Frequent NER,NER Tags
1,"Ive had tinnitus for 4 years, and it developed...",SYM,"[(tinnitus, SOUND, 8, 16), (infection, SYM, 70..."
2,I had a lot of fullness and loud ringing in my...,SOUND,"[(ringing, SOUND, 33, 40)]"
3,The ringing was only loud for maybe a month or...,SOUND,"[(ringing, SOUND, 4, 11), (infection, SYM, 73,..."
4,"Ive had a few ear infections since then, and i...",None,[]
5,"Fast forward to last week, I went and visited ...",None,[]


In [ ]:
df.shape

(555, 3)

In [ ]:
train_data = []  # Initialize the train_data list

for index, row in df.iterrows():
    sentence = row['Sentences']  # Get the sentence from the DataFrame
    ner_tags = row['NER Tags']  # Get the list of NER tags from the DataFrame

    entities = []  # Initialize the list of entities for the current sentence

    # Iterate through the NER tags and add them to the entities list
    for tag in ner_tags:
        word, ner_tag, start, end = tag
        entities.append((start, end, ner_tag))

    # Create a tuple for the current sentence with the entities
    train_data.append((sentence, {'entities': entities}))

# train_data is now in the format that spaCy's NER model expects
train_data[0]

('Ive had tinnitus for 4 years, and it developed after a really bad ear infection that developed because of a sinus infection',
 {'entities': [(8, 16, 'SOUND'),
   (70, 79, 'SYM'),
   (108, 113, 'SYM'),
   (114, 123, 'SYM')]})

In [ ]:
def check_overlapping_entities(train_data):
    for text, annotation in train_data:
        spans = [(start, end) for start, end, label in annotation['entities']]
        for i, (start1, end1) in enumerate(spans):
            for j, (start2, end2) in enumerate(spans):
                if i != j:
                    # Check if start1/end1 is between start2 and end2
                    if (start1 >= start2 and start1 < end2) or (end1 > start2 and end1 <= end2):
                        print(f"Overlap found in: {text}")
                        print(f"Entity 1: {text[start1:end1]}")
                        print(f"Entity 2: {text[start2:end2]}")
                        print("----")

check_overlapping_entities(train_data)


Overlap found in: I will sometimes go days without even checking things on the internet becuase I am doing 'better' - but then a spike will happen, or I wont sleep a few nights - and then I am once again on the internet in the rabbit hole of 'is this medication causing this, when did this happen - I ate this at x time or did this at x time and then tinnitus got worse/louder' - it's really turned into an OCD fixation and im trying to break that cycle, but some days it seems impossible.
Entity 1: medication
Entity 2: medication
----
Overlap found in: I will sometimes go days without even checking things on the internet becuase I am doing 'better' - but then a spike will happen, or I wont sleep a few nights - and then I am once again on the internet in the rabbit hole of 'is this medication causing this, when did this happen - I ate this at x time or did this at x time and then tinnitus got worse/louder' - it's really turned into an OCD fixation and im trying to break that cycle, but some

In [ ]:
def resolve_overlaps(entities):
    """
    Resolve overlapping entities by keeping the longer entity span.
    """
    # Sort entities by their length (longest entity first)
    entities = sorted(entities, key=lambda x: x[1] - x[0], reverse=True)
    resolved = []
    while entities:
        current_entity = entities.pop(0)
        current_start, current_end, _ = current_entity
        non_overlapping = [e for e in entities if e[1] <= current_start or e[0] >= current_end]
        resolved.append(current_entity)
        entities = non_overlapping
    return resolved



In [ ]:
def clean_train_data(train_data):
    """
    Clean training data by resolving overlapping entities.
    """
    cleaned_data = []
    for text, annotation in train_data:
        entities = annotation['entities']
        cleaned_entities = resolve_overlaps(entities)
        cleaned_data.append((text, {'entities': cleaned_entities}))
    return cleaned_data

# Use the function to clean the training data
cleaned_train_data = clean_train_data(train_data)


In [ ]:
import spacy
from spacy.training import offsets_to_biluo_tags

# Load a blank spacy model (ensure it's the same language as your texts)
nlp = spacy.blank("en")

def check_misaligned_entities(training_data):
    misaligned_texts = []

    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        entities = annotations.get("entities", [])
        tags = offsets_to_biluo_tags(doc, entities)

        if '-' in tags:
            misaligned_texts.append((text, entities))

    return misaligned_texts



misaligned = check_misaligned_entities(cleaned_train_data)
for text, entities in misaligned:
    print(f"Text: {text}\nEntities: {entities}\n")

# Here you can either manually correct the misaligned entities, or take further automated steps to do so.


Text: maybe it did go away?

Fast forward to mid october - I am having sleep issues, anxiety is increasing about it, I once again start to 'notice' tinnitus in my ear - I restart use of Xanax and Trazodone as needed to help me get some sleep, however, anxiety is still high

Doctor suggests I restart Prozac - and this is where my feed back loop of never ending anxiety starts
Entities: [(189, 198, 'MED'), (141, 149, 'SOUND'), (78, 85, 'NEM'), (245, 252, 'NEM'), (354, 361, 'NEM'), (267, 273, 'DOC'), (64, 69, 'SYM'), (328, 332, 'BOD')]

Text: Enough on that, lets get to what really matters, the solution:

Around the time my tinnitus became noticeable to me (like not being able to sit in quiet areas without immediately recognizing the ear ringing) I also started to developing clicking in my jaw whenever I opened my mouth
Entities: [(82, 90, 'SOUND'), (251, 254, 'BOD')]

Text: Doctors all said 'possible, but you would be first in 25 years to report that to me'

July-October 2022 - no meds - 

In [ ]:
import spacy

# Load the model you're using for training
nlp = spacy.load("en_core_web_sm")

def adjust_entities(text, entities, nlp):
    doc = nlp(text)
    adjusted_entities = []
    for start, end, label in entities:
        span = doc.char_span(start, end)
        if span:  # Make sure the span exists
            adjusted_entities.append((span.start_char, span.end_char, label))
    return adjusted_entities

# Adjust the entities in TRAIN_DATA
adjusted_data = []
for text, annotations in cleaned_train_data:
    adjusted_entities = adjust_entities(text, annotations["entities"], nlp)
    adjusted_data.append((text, {"entities": adjusted_entities}))

# Now use `adjusted_data` for training


In [ ]:
!pip install spacy
!python -m spacy download en  # Download the English model
!pip install spacy-lookups-data
!python -m spacy download en_core_web_sm



2023-10-22 14:28:43.246739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 14:28:44.417494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.2 MB/s eta 0:00:00
2023-10-22 14:29:12.255879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical ope

In [ ]:
'''nlp.pipe_names'''

NameError: ignored

In [ ]:
'''import spacy
from spacy.training.example import Example

# Load your existing model
nlp = spacy.load("en_core_web_sm")

# Create an optimizer for training
optimizer = nlp.create_optimizer()

# Your training data in the format [(text, annotations)]

# Train the model
for itn in range(10):
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5, losses=losses)
'''

In [ ]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example


In [ ]:
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner", last=True)


In [ ]:
for _, annotations in cleaned_train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])


In [ ]:
optimizer = nlp.initialize()
import random

# Number of times to train over the data
epochs = 30

for epoch in range(epochs):
    # Shuffle the training data
    random.shuffle(adjusted_data)
    losses = {}

    # Create batches of training data
    batches = minibatch(adjusted_data, size=compounding(4.0, 32.0, 1.001))

    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example, drop=0.5, losses=losses)

    print(f"Epoch {epoch + 1}, Losses: {losses}")


Epoch 1, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 1617.883809932072}
Epoch 2, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 573.2427610687641}
Epoch 3, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 484.9975976322293}
Epoch 4, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 409.97209068602814}
Epoch 5, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 392.91389916220857}
Epoch 6, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 357.3235717409633}
Epoch 7, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 345.5553098881831}
Epoch 8, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 326.5013499151951}
Epoch 9, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 304.2976431092799}
Epoch 10, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 287.946697278403}
Epoch 11, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 280.4658781982354}
Epoch 1

In [ ]:
import warnings

# Suppress specific spacy warnings
warnings.filterwarnings("ignore", category=UserWarning, module="spacy.pipeline.attributeruler")
warnings.filterwarnings("ignore", category=UserWarning, module="spacy.pipeline.lemmatizer")
doc = nlp("I've been experiencing vertigo for 3 years, which began following a severe throat infection that arose from a nasal congestion.")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


vertigo 23 30 SYM
severe 68 74 LEVEL
infection 82 91 SYM


In [ ]:
nlp.to_disk("path_to_saved_model_directory")


In [ ]:
import spacy
import pickle

# Save the model using spaCy's to_disk method
model_path = "./saved_model"
nlp.to_disk(model_path)

# Pickle the model path (if needed)
with open('model_path.pkl', 'wb') as f:
    pickle.dump(model_path, f)


In [ ]:
# Load model path from pickle
with open('model_path.pkl', 'rb') as f:
    model_path = pickle.load(f)

# Load spaCy model from disk
nlp_loaded = spacy.load(model_path)
